In [ ]:
#@title 🛠️ **Cell 1: Trinity Multi-Venv Infrastructure Setup v1.3** { display-mode: "form" }

# ============================================================================
# CELL 1: TRINITY MULTI-VENV INFRASTRUCTURE SETUP
# This version ensures all commands are non-interactive and streams all output.
# ============================================================================

import os
import subprocess
import sys
from pathlib import Path

# --- Basic Setup ---
PROJECT_ROOT = Path('/content/TrinityUI')
WEBUI_ROOT = Path('/content')
TRINITY_REPO_URL = 'https://github.com/remphanostar/TrinityUI.git'
TRINITY_BRANCH = 'UselessGemini'

def run_setup_command(command):
    """Runs a shell command and streams its output live."""
    print(f"🚀 Executing: {command}")
    process = subprocess.Popen(
        command,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding='utf-8',
        errors='replace'
    )

    # Print output line by line
    for line in iter(process.stdout.readline, ''):
        sys.stdout.write(f"  > {line}")
        sys.stdout.flush()

    # Wait for the process to complete and check the return code
    return_code = process.wait()
    if return_code != 0:
        raise subprocess.CalledProcessError(return_code, command)
    print(f"✅ Command successful.\n")

print("🚀 Starting Trinity Multi-Venv Setup...")

# 1. Ensure System Dependencies are present
print("\n[Step 1/3] Setting up system dependencies (this may take a moment)...")
try:
    # Use DEBIAN_FRONTEND=noninteractive to prevent any prompts
    os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

    run_setup_command("apt-get update -qq")
    run_setup_command("apt-get install -y -qq software-properties-common")
    # The 'y' flag is usually sufficient, but noninteractive frontend is safer
    run_setup_command("add-apt-repository -y ppa:deadsnakes/ppa")
    run_setup_command("apt-get install -y -qq python3.10 python3.10-venv python3.11 python3.11-venv git")

    print("   ✅ Python 3.10, 3.11, and Git are available.")
except Exception as e:
    print(f"   ❌ FAILED to install system dependencies: {e}")
    raise

# 2. Clone or Update the TrinityUI control repository
print(f"\n[Step 2/3] Setting up TrinityUI control repository from '{TRINITY_BRANCH}' branch...")
if (PROJECT_ROOT / '.git').is_dir():
    print("   - Existing repository found. Updating...")
    # Using run_setup_command to see output if there are issues
    run_setup_command(f"git --git-dir={PROJECT_ROOT / '.git'} --work-tree={PROJECT_ROOT} fetch")
    run_setup_command(f"git --git-dir={PROJECT_ROOT / '.git'} --work-tree={PROJECT_ROOT} reset --hard origin/{TRINITY_BRANCH}")
else:
    print("   - Cloning repository...")
    run_setup_command(f"git clone --branch {TRINITY_BRANCH} {TRINITY_REPO_URL} {PROJECT_ROOT}")
print("   ✅ TrinityUI repository is up to date.")


# 3. Execute the Venv Manager
print("\n[Step 3/3] Executing the Venv Manager to build isolated environments...")
print("="*60)
try:
    venv_manager_script = PROJECT_ROOT / 'scripts' / 'manage_venvs.py'
    run_setup_command(f"python3 {venv_manager_script}")

    print("="*60)
    print("✅ All isolated WebUI environments have been successfully set up!")

except Exception as e:
    print("="*60)
    print(f"❌ An error occurred during the Venv setup process: {e}")
    raise

# Set global variables for subsequent cells
globals()['PROJECT_ROOT'] = PROJECT_ROOT
globals()['WEBUI_ROOT'] = WEBUI_ROOT
globals()['trinity_setup_success'] = True

print("\n🎉 Infrastructure setup is complete. You may now proceed to Cell 2.")


In [2]:
#@title 🎛️ **Cell 2: Project Trinity v1.1.0 - Configuration Hub** { display-mode: "form" }

# ============================================================================
# CELL 2: PROJECT TRINITY - CONFIGURATION HUB (FINAL LAUNCHER)
# This cell correctly launches the configuration hub as a background process
# and prints the public Gradio URL for access.
# ============================================================================

import os
import sys
import importlib.util
from pathlib import Path
from datetime import datetime
import socket
import time

# --- Helper Functions ---
def find_free_port(start_port=7860, max_attempts=20):
    """Find a free port starting from start_port"""
    for i in range(max_attempts):
        port = start_port + i
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                continue
    return None

def log_to_unified(message, level="INFO", component="TRINITY-CELL-2"):
    """Log messages using the unified logging system from Cell 1"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_file = globals().get('LOG_FILE', Path.cwd() / 'trinity_unified.log')
    version = globals().get('VERSION', '1.1.0')
    log_entry = f"[{timestamp}] [v{version}] [{level}] [{component}] {message}\n"
    try:
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry)
    except Exception as e:
        print(f"Warning: Could not write to log file: {e}")
    if level in ["ERROR", "SUCCESS"] or "Trinity" in message:
        print(f"[{level}] {message}")

# --- Infrastructure Check ---
def check_trinity_infrastructure():
    if 'PROJECT_ROOT' not in globals() or not globals()['PROJECT_ROOT'].exists():
        return False, "PROJECT_ROOT not found. Please run Cell 1 first."
    return True, "Infrastructure OK"

infrastructure_ok, infrastructure_msg = check_trinity_infrastructure()
if not infrastructure_ok:
    print(f"❌ CRITICAL: {infrastructure_msg}")
    raise RuntimeError(infrastructure_msg)

# --- Main Execution ---
PROJECT_ROOT = globals()['PROJECT_ROOT']
sys.path.insert(0, str(PROJECT_ROOT))

log_to_unified("Preparing Trinity Configuration Hub...")

# Find a free port
config_port = find_free_port(7860, 20)
if config_port is None:
    log_to_unified("Could not find a free port, using fallback 7861", "WARNING")
    config_port = 7861
else:
    log_to_unified(f"Found free port for Config Hub: {config_port}", "SUCCESS")

os.environ['TRINITY_CONFIG_PORT'] = str(config_port)

# Load and execute the configuration hub script
try:
    # Ensure we are using the corrected script
    config_script_path = PROJECT_ROOT / 'scripts' / 'configuration_hub_new.py'

    if config_script_path.exists():
        log_to_unified(f"Executing configuration hub from: {config_script_path}")

        # Use importlib to load the script as a module
        spec = importlib.util.spec_from_file_location("config_hub_module", config_script_path)
        config_hub_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(config_hub_module)

        # Explicitly call the function that launches the Gradio app.
        # This will now print the URL and keep the cell running.
        if hasattr(config_hub_module, 'launch_trinity_configuration_hub'):
            log_to_unified("Launching Gradio interface...")
            # This function now contains the launch() call with share=True
            config_hub_module.launch_trinity_configuration_hub()
        else:
            raise RuntimeError("Could not find 'launch_trinity_configuration_hub' function in the script.")

    else:
        log_to_unified("Configuration Hub script (configuration_hub_new.py) not found!", "ERROR")
        print("❌ ERROR: scripts/configuration_hub_new.py not found.")

except Exception as e:
    log_to_unified(f"Configuration hub execution failed: {e}", "ERROR")
    print(f"❌ ERROR: Failed to launch the Configuration Hub. Details: {e}")
    raise


[INFO] Preparing Trinity Configuration Hub...
[SUCCESS] Found free port for Config Hub: 7860
[INFO] Executing configuration hub from: /content/TrinityUI/scripts/configuration_hub_new.py
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e172f97fdac22f6286.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
#@title 🚀 **Cell 3: Project Trinity v1.1.0 - Asset Download & Launch** { display-mode: "form" }

# ============================================================================
# CELL 3: PROJECT TRINITY - UNIFIED LAUNCH ENGINE
# This cell calls a single, definitive script to handle the launch process.
# ============================================================================

import os
import sys
from pathlib import Path
import importlib.util

# --- Infrastructure Check ---
def check_trinity_infrastructure():
    if 'PROJECT_ROOT' not in globals() or not globals()['PROJECT_ROOT'].exists():
        return False, "PROJECT_ROOT not found. Please run Cell 1 first."
    if not (globals()['PROJECT_ROOT'] / 'trinity_config.json').exists():
        return False, "trinity_config.json not found. Please run Cell 2 first."
    return True, "Infrastructure and configuration OK."

infrastructure_ok, infrastructure_msg = check_trinity_infrastructure()
if not infrastructure_ok:
    print(f"❌ CRITICAL: {infrastructure_msg}")
    raise RuntimeError(infrastructure_msg)

# --- Main Execution ---
PROJECT_ROOT = globals()['PROJECT_ROOT']
sys.path.insert(0, str(PROJECT_ROOT))
print(f"✅ Infrastructure validated. Project root: {PROJECT_ROOT}")
print("   Preparing to execute the main launch engine...")

try:
    # We now point to a single, definitive launch script.
    script_path = PROJECT_ROOT / 'scripts' / 'execute_launch.py'

    if not script_path.exists():
        raise FileNotFoundError("Could not find the main launch script: scripts/execute_launch.py")

    # Import the main function from the script
    spec = importlib.util.spec_from_file_location("launcher_module", script_path)
    launcher_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(launcher_module)

    # This call will be BLOCKING and will stream all output here.
    launcher_module.main()

except KeyboardInterrupt:
    print("\n\n✅ Execution stopped by user.")
except Exception as e:
    print(f"\n\n❌ An error occurred during execution: {e}")
    raise


✅ Infrastructure validated. Project root: /content/TrinityUI
   Preparing to execute the main launch engine...
[INFO] === Trinity Venv-Aware Execution Engine Started ===
[SUCCESS] Executing command: /content/A1111/venv/bin/python launch.py --share --xformers

🚀 LAUNCHING A1111 FROM ITS VENV...
Cloning into '/content/A1111/repositories/stable-diffusion-webui-assets'...
Cloning into '/content/A1111/repositories/stable-diffusion-stability-ai'...
Cloning into '/content/A1111/repositories/generative-models'...
Cloning into '/content/A1111/repositories/k-diffusion'...
Cloning into '/content/A1111/repositories/BLIP'...
/content/A1111/venv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Python 3.10.12 (main, May 27 2025, 17:12:29) [GCC 11.4.0]
Version: v1.10.1
Commit hash: 82a973c0

In [9]:
!cd /content/A1111 && /content/A1111/venv/bin/python launch.py --share --enable-insecure-extension-access --xformers --api --theme Default

Python 3.10.12 (main, May 27 2025, 17:12:29) [GCC 11.4.0]
Version: v1.10.1
Commit hash: 82a973c04367123ae98bd9abdf80d9eda9b910e2
Launching Web UI with arguments: --share --enable-insecure-extension-access --xformers --api --theme Default
Traceback (most recent call last):
  File "/content/A1111/launch.py", line 48, in <module>
    main()
  File "/content/A1111/launch.py", line 44, in main
    start()
  File "/content/A1111/modules/launch_utils.py", line 465, in start
    import webui
  File "/content/A1111/webui.py", line 13, in <module>
    initialize.imports()
  File "/content/A1111/modules/initialize.py", line 17, in imports
    import pytorch_lightning  # noqa: F401
  File "/content/A1111/venv/lib/python3.10/site-packages/pytorch_lightning/__init__.py", line 35, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
  File "/content/A1111/venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/__init__.py", line 14, in <module>
    from pytorch_lightni